# 1. Data Preparation
 1.1 Load data \
 1.2 Add a penalty info related columns \
 1.3 Filter dataframe \
 1.4 Update team info frame with power play and penalty kill count

In [1]:
import numpy as np
import pandas as pd
from pandasql import sqldf
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('data/Rotman MMA Summer Datathon NWHL.csv')
df_game_info = pd.read_csv('data/game_info.csv', sep="\t")
df_event_info = pd.read_csv('data/event_info.csv', sep="\t")
df_player_info = pd.read_csv('data/player_info.csv', sep="\t")
df_team_info = pd.read_csv('data/team_info.csv', sep="\t")


In [3]:
df_team_info.head()

,Team_Name,num_home_game,num_away_game,num_game
0,Boston Pride,2,5,7
1,Buffalo Beauts,4,2,6
2,Connecticut Whale,2,2,4
3,Metropolitan Riveters,2,1,3
4,Minnesota Whitecaps,1,3,4


In [4]:
len(df)

26882

In [5]:
df.head()

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,...,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2021-01-23,Minnesota Whitecaps,Boston Pride,1,20:00,5,5,0,0,Boston Pride,...,Faceoff Win,100,43,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN
1,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:58,5,5,0,0,Boston Pride,...,Puck Recovery,107,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:57,5,5,0,0,Boston Pride,...,Zone Entry,125,28,Carried,NaN,NaN,NaN,Maddie Rowe,NaN,NaN
3,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:55,5,5,0,0,Boston Pride,...,Shot,131,28,Snapshot,On Net,t,f,NaN,NaN,NaN
4,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:53,5,5,0,0,Boston Pride,...,Faceoff Win,169,21,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN


In [6]:
# add a penalty_period column to distinguish different penalty periods
df["penalty_period"] = np.where(df["Event"]=="Penalty Taken", 1, 0)
df["penalty_period"] = df["penalty_period"].cumsum()

In [7]:
df.query("Event == 'Penalty Taken'")

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,...,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2,penalty_period
210,2021-01-23,Minnesota Whitecaps,Boston Pride,1,12:48,5,5,0,0,Minnesota Whitecaps,...,83,46,Tripping,NaN,NaN,NaN,Taylor Wenczkowski,NaN,NaN,1
295,2021-01-23,Minnesota Whitecaps,Boston Pride,1,9:57,5,5,0,0,Minnesota Whitecaps,...,10,12,Roughing,NaN,NaN,NaN,Carlee Turner,NaN,NaN,2
458,2021-01-23,Minnesota Whitecaps,Boston Pride,1,4:18,5,5,1,1,Minnesota Whitecaps,...,5,30,Hooking,NaN,NaN,NaN,Meghara McManus,NaN,NaN,3
503,2021-01-23,Minnesota Whitecaps,Boston Pride,1,2:15,5,5,1,1,Boston Pride,...,162,83,Interference,NaN,NaN,NaN,NaN,NaN,NaN,4
682,2021-01-23,Minnesota Whitecaps,Boston Pride,2,15:30,5,5,1,1,Minnesota Whitecaps,...,94,5,Hooking,NaN,NaN,NaN,Tereza Vanisova,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26442,2021-02-01,Buffalo Beauts,Boston Pride,3,17:12,5,5,1,4,Boston Pride,...,190,64,Tripping,NaN,NaN,NaN,Whitney Dove,NaN,NaN,140
26582,2021-02-01,Buffalo Beauts,Boston Pride,3,11:27,5,5,1,5,Buffalo Beauts,...,84,67,Tripping,NaN,NaN,NaN,Meghara McManus,NaN,NaN,141
26619,2021-02-01,Buffalo Beauts,Boston Pride,3,9:55,5,5,1,6,Boston Pride,...,137,84,Holding,NaN,NaN,NaN,Autumn MacDougall,NaN,NaN,142
26706,2021-02-01,Buffalo Beauts,Boston Pride,3,6:27,5,5,1,7,Buffalo Beauts,...,151,82,Interference,NaN,NaN,NaN,Taylor Wenczkowski,NaN,NaN,143


In [8]:
# choose power play / penalty kill moment by compare the number of players on the court
df = df[df["Home Team Skaters"] != df["Away Team Skaters"]]

In [9]:
# change column names for sql
df.columns = df.columns.str.strip().str.replace(' ', '_')

In [10]:
len(df)

5898

In [11]:
df.head(5)

,game_date,Home_Team,Away_Team,Period,Clock,Home_Team_Skaters,Away_Team_Skaters,Home_Team_Goals,Away_Team_Goals,Team,...,X_Coordinate,Y_Coordinate,Detail_1,Detail_2,Detail_3,Detail_4,Player_2,X_Coordinate_2,Y_Coordinate_2,penalty_period
211,2021-01-23,Minnesota Whitecaps,Boston Pride,1,12:48,4,5,0,0,Boston Pride,...,169,65,Forehand,NaN,NaN,NaN,Jonna Curtis,NaN,NaN,1
212,2021-01-23,Minnesota Whitecaps,Boston Pride,1,12:47,4,5,0,0,Boston Pride,...,144,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
213,2021-01-23,Minnesota Whitecaps,Boston Pride,1,12:45,4,5,0,0,Boston Pride,...,130,83,Direct,NaN,NaN,NaN,Lauren Kelly,131.0,59.0,1
214,2021-01-23,Minnesota Whitecaps,Boston Pride,1,12:43,4,5,0,0,Boston Pride,...,136,32,Direct,NaN,NaN,NaN,Taylor Turnquist,159.0,4.0,1
215,2021-01-23,Minnesota Whitecaps,Boston Pride,1,12:39,4,5,0,0,Boston Pride,...,162,2,Direct,NaN,NaN,NaN,Lauren Kelly,129.0,49.0,1


In [12]:
# add a column "is_power_play_event" to indicate whether the event happening belongs to a power play or penalty kill
# 1 for power play, 
# 0 for penalty kill
df['is_power_play_event'] = np.where(
    ((df['Home_Team'] == df['Team']) & (df["Home_Team_Skaters"] > df["Away_Team_Skaters"])) | 
    ((df['Away_Team'] == df['Team']) & (df["Home_Team_Skaters"] < df["Away_Team_Skaters"])), 
    1, 0)

In [13]:
# add two columns to indicate which team is in power play
df["Power_Play_Team"] = np.where(df["Home_Team_Skaters"] > df["Away_Team_Skaters"], df["Home_Team"], df["Away_Team"])
df["Penalty_Kill_Team"] = np.where(df["Home_Team_Skaters"] > df["Away_Team_Skaters"], df["Away_Team"], df["Home_Team"])


In [14]:
df.tail()

,game_date,Home_Team,Away_Team,Period,Clock,Home_Team_Skaters,Away_Team_Skaters,Home_Team_Goals,Away_Team_Goals,Team,...,Detail_2,Detail_3,Detail_4,Player_2,X_Coordinate_2,Y_Coordinate_2,penalty_period,is_power_play_event,Power_Play_Team,Penalty_Kill_Team
26877,2021-02-01,Buffalo Beauts,Boston Pride,3,0:06,4,5,1,7,Buffalo Beauts,...,NaN,NaN,NaN,NaN,NaN,NaN,144,0,Boston Pride,Buffalo Beauts
26878,2021-02-01,Buffalo Beauts,Boston Pride,3,0:06,4,5,1,7,Buffalo Beauts,...,NaN,NaN,NaN,Cassidy MacPherson,29.0,82.0,144,0,Boston Pride,Buffalo Beauts
26879,2021-02-01,Buffalo Beauts,Boston Pride,3,0:05,4,5,1,7,Buffalo Beauts,...,NaN,NaN,NaN,NaN,NaN,NaN,144,0,Boston Pride,Buffalo Beauts
26880,2021-02-01,Buffalo Beauts,Boston Pride,3,0:03,4,5,1,7,Boston Pride,...,NaN,NaN,NaN,NaN,NaN,NaN,144,1,Boston Pride,Buffalo Beauts
26881,2021-02-01,Buffalo Beauts,Boston Pride,3,0:03,4,5,1,7,Boston Pride,...,NaN,NaN,NaN,Lexie Laing,155.0,6.0,144,1,Boston Pride,Buffalo Beauts


In [15]:
# count the number of power play and penalty kill period for each team
df_power_play_count = df.groupby("Power_Play_Team").penalty_period.nunique().reset_index().rename(columns={"penalty_period": "power_play_count"})
df_penalty_count = df.groupby("Penalty_Kill_Team").penalty_period.nunique().reset_index().rename(columns={"penalty_period": "penalty_kill_count"})


In [16]:
df_power_play_count

,Power_Play_Team,power_play_count
0,Boston Pride,47
1,Buffalo Beauts,22
2,Connecticut Whale,12
3,Metropolitan Riveters,12
4,Minnesota Whitecaps,22
5,Toronto Six,32


In [17]:
# update team info frame with power play and penalty kill count
df_team_info = pd.merge(df_team_info, df_power_play_count, left_on="Team_Name", right_on="Power_Play_Team", )
df_team_info = pd.merge(df_team_info, df_penalty_count, left_on="Team_Name", right_on="Penalty_Kill_Team", )
df_team_info = df_team_info[["Team_Name", "num_home_game", "num_away_game", "num_game", "power_play_count", "penalty_kill_count"]]
df_team_info

,Team_Name,num_home_game,num_away_game,num_game,power_play_count,penalty_kill_count
0,Boston Pride,2,5,7,47,27
1,Buffalo Beauts,4,2,6,22,30
2,Connecticut Whale,2,2,4,12,19
3,Metropolitan Riveters,2,1,3,12,13
4,Minnesota Whitecaps,1,3,4,22,25
5,Toronto Six,4,2,6,32,33


# 2. Find Penalty Kill Specialists
 The key skills for a penalty killer are takeaways and precise passes. 

### 2.1 Find the players with most takeaways 
<ol>
  <li>in total</li>
  <li>per game </li>
  <li>per penalty kill</li>
</ol>
   
   
   

In [18]:
df_penalty_kill = df[df['is_power_play_event'] == 0]

In [19]:
df_penalty_kill_takeaways = df_penalty_kill[df_penalty_kill['Event'] == 'Takeaway']

In [20]:
len(df_penalty_kill_takeaways)

124

In [21]:
# count the number of takeaways for each player in the penalty kill
df_penalty_kill_takeaways_count = df_penalty_kill_takeaways["Player"].groupby(df_penalty_kill_takeaways["Player"]).count().sort_values(ascending=False).reset_index(name="takeaway_count")

In [22]:
df_penalty_kill_takeaways_count.head()

,Player,takeaway_count
0,Meghara McManus,8
1,Taylor Woods,7
2,Natalie Marcuzzi,5
3,Sydney Baldwin,5
4,Taylor Turnquist,4


In [23]:
df_player_info.head()

,Team,Player,num_game
0,Minnesota Whitecaps,Jillian Dempsey,4
1,Minnesota Whitecaps,McKenna Brand,4
2,Minnesota Whitecaps,Tereza Vanisova,4
3,Minnesota Whitecaps,Samantha Davis,4
4,Minnesota Whitecaps,Stephanie Anderson,4


In [24]:
df_team_info.head()

,Team_Name,num_home_game,num_away_game,num_game,power_play_count,penalty_kill_count
0,Boston Pride,2,5,7,47,27
1,Buffalo Beauts,4,2,6,22,30
2,Connecticut Whale,2,2,4,12,19
3,Metropolitan Riveters,2,1,3,12,13
4,Minnesota Whitecaps,1,3,4,22,25


In [25]:
# join the three tables above to figure out the takeaway stats of each player in penalty kill

df_penalty_kill_takeaways_player_stats = pd.merge(df_penalty_kill_takeaways_count, df_player_info, on="Player")
df_penalty_kill_takeaways_player_stats = pd.merge(df_penalty_kill_takeaways_player_stats, df_team_info, left_on="Team", right_on="Team_Name")
df_penalty_kill_takeaways_player_stats = df_penalty_kill_takeaways_player_stats.assign(takeaway_per_game = lambda x: x["takeaway_count"] / x["num_game_x"],
                                                                                       takeaway_per_penalty_kill = lambda x: x["takeaway_count"] / x["penalty_kill_count"])


In [26]:
df_penalty_kill_takeaways_player_stats

,Player,takeaway_count,Team,num_game_x,Team_Name,num_home_game,num_away_game,num_game_y,power_play_count,penalty_kill_count,takeaway_per_game,takeaway_per_penalty_kill
0,Meghara McManus,8,Minnesota Whitecaps,4,Minnesota Whitecaps,1,3,4,22,25,2.000000,0.320000
1,Sydney Baldwin,5,Minnesota Whitecaps,4,Minnesota Whitecaps,1,3,4,22,25,1.250000,0.200000
2,Taylor Turnquist,4,Minnesota Whitecaps,4,Minnesota Whitecaps,1,3,4,22,25,1.000000,0.160000
3,Stephanie Anderson,4,Minnesota Whitecaps,4,Minnesota Whitecaps,1,3,4,22,25,1.000000,0.160000
4,Kaleigh Fratkin,3,Minnesota Whitecaps,4,Minnesota Whitecaps,1,3,4,22,25,0.750000,0.120000
...,...,...,...,...,...,...,...,...,...,...,...,...
181,Meghan Lorence,1,Metropolitan Riveters,3,Metropolitan Riveters,2,1,3,12,13,0.333333,0.076923
182,Megan Quinn,1,Metropolitan Riveters,3,Metropolitan Riveters,2,1,3,12,13,0.333333,0.076923
183,Emma Greco,1,Metropolitan Riveters,3,Metropolitan Riveters,2,1,3,12,13,0.333333,0.076923
184,Emma Woods,1,Metropolitan Riveters,3,Metropolitan Riveters,2,1,3,12,13,0.333333,0.076923


In [27]:
df_penalty_kill_takeaways_player_stats = df_penalty_kill_takeaways_player_stats[["Player", "Team", "takeaway_count", "num_game_x", "takeaway_per_game", "penalty_kill_count", "takeaway_per_penalty_kill"]]

In [28]:
df_penalty_kill_takeaways_player_stats[["takeaway_per_game", "takeaway_per_penalty_kill"]] = df_penalty_kill_takeaways_player_stats[["takeaway_per_game", "takeaway_per_penalty_kill"]].applymap("{0:.2f}".format)

/var/folders/m3/4yh806w92fdgcn0bk16ql7nw0000gn/T/ipykernel_61496/975769985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_penalty_kill_takeaways_player_stats[["takeaway_per_game", "takeaway_per_penalty_kill"]] = df_penalty_kill_takeaways_player_stats[["takeaway_per_game", "takeaway_per_penalty_kill"]].applymap("{0:.2f}".format)


In [29]:
df_penalty_kill_takeaways_player_stats

,Player,Team,takeaway_count,num_game_x,takeaway_per_game,penalty_kill_count,takeaway_per_penalty_kill
0,Meghara McManus,Minnesota Whitecaps,8,4,2.00,25,0.32
1,Sydney Baldwin,Minnesota Whitecaps,5,4,1.25,25,0.20
2,Taylor Turnquist,Minnesota Whitecaps,4,4,1.00,25,0.16
3,Stephanie Anderson,Minnesota Whitecaps,4,4,1.00,25,0.16
4,Kaleigh Fratkin,Minnesota Whitecaps,3,4,0.75,25,0.12
...,...,...,...,...,...,...,...
181,Meghan Lorence,Metropolitan Riveters,1,3,0.33,13,0.08
182,Megan Quinn,Metropolitan Riveters,1,3,0.33,13,0.08
183,Emma Greco,Metropolitan Riveters,1,3,0.33,13,0.08
184,Emma Woods,Metropolitan Riveters,1,3,0.33,13,0.08
